# 1. Approach to Optimisation
- Focus on the bottlenecks
- Consider Data/ Task parallelism
- Use optimised compiled tools
- Timing bash
    - { time [application] ; } 2> time.txt

# 2. Workflow

## 2.1 QC

## 2.2 Trimming
* Common tools are fastp, Trimmomatic, Cutadapt, and Trim_Galore [1]
* But most common are fastp and Trim_galore as fastp is fast and simple while trim_galore also generate QC report with trimming and filtering (integrate cutadpt and fastqc into a single process)
* fastp is faster than trim_galore

## 2.3 Mapping and Quantification
* By Align to genome + Quantify post alignment OR Quasi-mapping Approach


### 2.3.1 Alignment
* Bowtie, Bowtie2, TopHat, TopHat2, HISAT, HISAT2, STAR are commonly used tools for alignment
* HISAT2 demonstrated the quickest processing time
* NOTE: Different software give different alignment rate

### 2.3.2 Quantification
* Alignment-based transcript quantification tools: featureCounts, HTseq, RSEM
    * featureCounts is fast
    * RSEM can simultaneously perform both alignment and quantification tasks
* alignment-free transcript quantification tools, including Salmon, Kallisto
* NOTE: Previous studies have demonstrated that alignment-based quantification tools yield higher accuracy in results

## 2.4 Compiling abundance into count matrix

# 3. Original seq bash script

In [ ]:
#!/bin/bash
#PBS -N RNAseq
#PBS -j oe
#PBS -l select=1:ncpus=8:mem=40gb
#PBS -l walltime=12:00:00

#################### DO NOT MODIFY ###################################
source /etc/profile.d/rec_modules.sh #make module list available
bash
. ~/.bashrc

conda activate my_env
#################### DO NOT MODIFY ###################################

# define source dir
SOURCE="/hpctmp/e1324266/5004/raw/download/fastq_files"
# define ref dir
REF_PATH="/hpctmp/e1324266/5004/ref/paper"
# define output dir
OUTPUT="/hpctmp/e1324266/5004/seq_bash"
# define results
RESULTS="/hpctmp/e1324266/5004/seq_bash/results"

# define ref files
genome="${REF_PATH}/GRCh37.p12.genome.fa"
gtf="${REF_PATH}/gencode.v18.annotation.gtf"
transcript="${REF_PATH}/gencode.v18.pc_transcripts.fa"
star_index="${REF_PATH}/star_index"

# deinfe dir
fastqc_dir="${RESULTS}/results/01-fastqc"
trim_dir="${RESULTS}/results/03-trim"
align_dir="${RESULTS}/results/04-align"
quant_dir="${RESULTS}/results/05-quant-align"

# Make directory
mkdir -p "${RESULTS}"
mkdir -p "${start_index}"
mkdir -p "${fastqc_dir}"
mkdir -p "${trim_dir}"
mkdir -p "${align_dir}"
mkdir -p "${quant_dir}"

source /app1/ebenv
module load STAR/2.7.10b-GCC-11.3.0
echo "Starting creating STAR index"
## Creating index
{ time STAR --runMode genomeGenerate \
    --runThreadN 8 \
    --genomeDir "$star_index" \
    --genomeFastaFiles "$genome" \
    --sjdbGTFfile "$gtf" \
    --sjdbOverhang 100; } 2> ${OUTPUT}/profiling_star_index.txt
module unload STAR/2.7.10b-GCC-11.3.0

time (
    for fq in ${SOURCE}/*_1.fastq.gz
    do
            # grab base of filename for naming outputs
            sample_id=$(basename $fq _1.fastq.gz)
            echo "Sample name is $sample_id"

    ##Paired fastq files
            fq1=${SOURCE}/${sample_id}_1.fastq.gz
            fq2=${SOURCE}/${sample_id}_2.fastq.gz

    # set up output filenames and locations
            #the trimming fastq format might be slight differnt due to different versions of the software
            trim_out_fq1=${trim_dir}/${sample_id}_val_1.fq.gz
            trim_out_fq2=${trim_dir}/${sample_id}_val_2.fq.gz

            # BAM file
            bam=${align_dir}/${sample_id}_Aligned.toTranscriptome.out.bam

            #starting
            echo "Processing sample $sample_id"
    # Run FastQC
            source /app1/ebenv
            module load FastQC/0.12.1-Java-11
            echo "Starting QC for $sample_id"
           { time fastqc -t 8 -o "$fastqc_dir" -f fastq -q "$fq1" "$fq2"; } 2>> ${RESULTS}/profiling_fastqc.txt
    # Run Trimming
            source /app1/ebenv
            module unload FastQC/0.12.1-Java-11 
            module load Trim_Galore/0.6.10-GCCcore-11.3.0
            echo "Starting Trimming for $sample_id"
           { time trim_galore --cores 8 --gzip --paired "$fq1" "$fq2" -o "$trim_dir" --basename $sample_id; } 2>> ${RESULTS}/profiling_trim_run.txt

    # Run STAR
            source /app1/ebenv
            module unload Trim_Galore/0.6.10-GCCcore-11.3.0
            module load STAR/2.7.10b-GCC-11.3.0
            echo "Starting run star read mapping for $sample_id"
           # Run alignment with STAR
           { time STAR --runThreadN 8 \
                --genomeDir "$star_index" \
                --readFilesIn "$trim_out_fq1" "$trim_out_fq2" \
                --readFilesCommand zcat \
                --outFileNamePrefix "${align_dir}/${sample_id}_" \
                --outFilterType BySJout \
                --outSAMtype BAM SortedByCoordinate \
                --outSAMattributes Standard \
                --sjdbGTFfile "$gtf" \
                --quantMode TranscriptomeSAM; } 2>> ${RESULTS}/profiling_star_align_run.txt

    # RUN SALMON QUANT     
            source /app1/ebenv
            module unload STAR/2.7.10b-GCC-11.3.0
            module load Salmon/1.9.0-GCC-11.3.0
            echo "Starting salmon quant for $sample_id"
            { time salmon quant --threads 8 \
                --targets $transcript \
                --libType A \
                --gencode \
                --output ${quant_dir}/${sample_id} \
                --alignments $bam \
                --seqBias \
                --useVBOpt; } 2>> ${RESULTS}/profiling_salmon_quant_run.txt   

    done
) 2> ${RESULTS}/profiling_loop.txt

# 4. Optim program seq bash script
Package optimisation
- fastqc
- fastp
- histat2
- featurecounts

In [ ]:
# HISAT2 ref builder
#!/bin/bash
#PBS -N hisat2
#PBS -l walltime=04:00:00
#PBS -l select=1:ncpus=4:mem=16gb

# Load required modules
source /app1/ebenv
module load HISAT2/2.2.1-gompi-2023a

# Change to the directory where the job was submitted
cd $PBS_O_WORKDIR

REF_PATH="/hpctmp/e1324266/5004/ref/paper"
OUTPUT="/hpctmp/e1324266/5004/ref/hisat2_index/genome"
mkdir -p ${OUTPUT}
genome="${REF_PATH}/GRCh37.p12.genome.fa"
hisat2-build ${genome} ${OUTPUT}

## 4.1 unpararellised workflow

In [ ]:
#!/bin/bash
#PBS -N rna_optim_1
#PBS -j oe
#PBS -l select=1:ncpus=12:mem=24gb
#PBS -l walltime=12:00:00

# Change to the directory where the job was submitted
cd $PBS_O_WORKDIR

# define source dir
SOURCE="/hpctmp/e1324266/5004/raw/download/fastq_files"
# define ref dir
REF_PATH="/hpctmp/e1324266/5004/ref/paper"
# define results
RESULTS="/hpctmp/e1324266/5004/seq_optim_bash"

# define ref files
genome="${REF_PATH}/GRCh37.p12.genome.fa"
gtf="${REF_PATH}/gencode.v18.annotation.gtf"
transcript="${REF_PATH}/gencode.v18.pc_transcripts.fa"

# define hisat2 index files
hisat2_index="/hpctmp/e1324266/5004/ref/hisat2_index/genome"

# deinfe dir
fastqc_dir="${RESULTS}/results/01-fastqc"
trim_dir="${RESULTS}/results/03-trim"
align_dir="${RESULTS}/results/04-align"
quant_dir="${RESULTS}/results/05-quant-align"

# Make directory
mkdir -p "${fastqc_dir}"
mkdir -p "${trim_dir}"
mkdir -p "${align_dir}"
mkdir -p "${quant_dir}"

time (
    for fq in ${SOURCE}/*_1.fastq.gz
    do
    # grab base of filename for naming outputs
        sample_id=$(basename $fq _1.fastq.gz)
        echo "Sample name is $sample_id"

    ##Paired fastq files
        fq1=${SOURCE}/${sample_id}_1.fastq.gz
        fq2=${SOURCE}/${sample_id}_2.fastq.gz

    # set up output filenames and locations
    #the trimming fastq format might be slight differnt due to different versions of the software
        trim_out_fq1=${trim_dir}/${sample_id}_trimmed_1.fq.gz
        trim_out_fq2=${trim_dir}/${sample_id}_trimmed_2.fq.gz

    # SAM file
        sam=${align_dir}/${sample_id}.sam

    #starting
        echo "Processing sample $sample_id"
        
  # Run FastQC
        source /app1/ebenv
        module load FastQC/0.12.1-Java-11
        echo "Starting QC for $sample_id"
        { time fastqc -t 4 -o "$fastqc_dir" -f fastq -q "$fq1" "$fq2"; } 2>> ${RESULTS}/optim1_fastqc.txt
    # Run Trimming
        source /app1/ebenv
        module unload FastQC/0.12.1-Java-11 
        module load fastp/0.23.2-GCC-11.3.0
        echo "Starting Trimming for $sample_id"
        { time fastp --thread 6 \
                    --in1 "$fq1" \
                    --in2 "$fq2" \
                    --out1 ${trim_dir}/${sample_id}_trimmed_1.fq.gz \
                    --out2 ${trim_dir}/${sample_id}_trimmed_2.fq.gz \
                    --detect_adapter_for_pe \
                    --qualified_quality_phred 20; } 2>> ${RESULTS}/optim1_trim_run.txt
        # Run HISAT2
        source /app1/ebenv
        module unload fastp/0.23.2-GCC-11.3.0
        module load HISAT2/2.2.1-gompi-2023a
        { time hisat2 -p 12 -x ${hisat2_index} -1 "${trim_out_fq1}" -2 "${trim_out_fq2}" -S "${align_dir}/${sample_id}.sam"; } 2>> ${RESULTS}/optim1_hisat2_align_run.txt

        # RUN featureCounts QUANT     
        source /app1/ebenv
        module unload HISAT2/2.2.1-gompi-2023a
        module load Subread/2.0.4-GCC-11.3.0
        echo "Starting featurecounts quant for $sample_id"
        { time featureCounts -p --countReadPairs -a ${gtf} -T 12 -t exon -g gene_id -M -o ${quant_dir}/${sample_id}_counts.txt ${sam}; } 2>> ${RESULTS}/optim1_subread_quant_run.txt   

    done
) 2> ${RESULTS}/optim1_loop.txt


## 4.2 parallelised
* task parallelised --> fastqc in backgorund with "&" "wait"

In [ ]:
#!/bin/bash

# define source dir
SOURCE="/hpctmp/e1324266/5004/raw/download/fastq_files"
# define ref dir
REF_PATH="/hpctmp/e1324266/5004/ref/paper"
# define results
RESULTS="/hpctmp/e1324266/5004/par_optim_bash"

# define ref files
genome="${REF_PATH}/GRCh37.p12.genome.fa"
gtf="${REF_PATH}/gencode.v18.annotation.gtf"
transcript="${REF_PATH}/gencode.v18.pc_transcripts.fa"

# define hisat2 index files
hisat2_index="/hpctmp/e1324266/5004/ref/hisat2_index/genome"

# deinfe dir
fastqc_dir="${RESULTS}/results/01-fastqc"
trim_dir="${RESULTS}/results/03-trim"
align_dir="${RESULTS}/results/04-align"
quant_dir="${RESULTS}/results/05-quant-align"

# Make directory
mkdir -p "${fastqc_dir}"
mkdir -p "${trim_dir}"
mkdir -p "${align_dir}"
mkdir -p "${quant_dir}"


for fq in ${SOURCE}/*_1.fastq.gz
do
# grab base of filename for naming outputs
sample_id=$(basename $fq _1.fastq.gz)

##Paired fastq files
fq1=${SOURCE}/${sample_id}_1.fastq.gz
fq2=${SOURCE}/${sample_id}_2.fastq.gz

# set up output filenames and locations
#the trimming fastq format might be slight differnt due to different versions of the software
trim_out_fq1=${trim_dir}/${sample_id}_trimmed_1.fq.gz
trim_out_fq2=${trim_dir}/${sample_id}_trimmed_2.fq.gz

# SAM file
sam=${align_dir}/${sample_id}.sam

qsub << EOF

#!/bin/bash
#PBS -N rna_optim_2_${sample_id}
#PBS -V
#PBS -j oe
#PBS -l select=1:ncpus=6:mem=12gb
#PBS -l walltime=12:00:00

#################### DO NOT MODIFY ###################################

# Change to the directory where the job was submitted
cd $PBS_O_WORKDIR

#################### DO NOT MODIFY ###################################

time (
    echo "Sample name is $sample_id"

    #starting
    echo "Processing sample $sample_id"
    # Run FastQC
    source /app1/ebenv
    module load FastQC/0.12.1-Java-11
    echo "Starting QC for $sample_id"
    { time fastqc -t 4 -o ${fastqc_dir} -f fastq -q ${fq1} ${fq2}; } 2>> ${RESULTS}/optim2_fastqc.txt &
    # Run Trimming
    source /app1/ebenv
    module unload FastQC/0.12.1-Java-11 
    module load fastp/0.23.2-GCC-11.3.0
    echo "Starting Trimming for $sample_id"
    { time fastp --thread 4 \
                --in1 ${fq1} \
                --in2 ${fq2} \
                --out1 ${trim_dir}/${sample_id}_trimmed_1.fq.gz \
                --out2 ${trim_dir}/${sample_id}_trimmed_2.fq.gz \
                --detect_adapter_for_pe \
                --qualified_quality_phred 20; } 2>> ${RESULTS}/optim2_trim_run.txt
    
    # Run HISAT2
    source /app1/ebenv
    module unload fastp/0.23.2-GCC-11.3.0
    module load HISAT2/2.2.1-gompi-2023a
    { time hisat2 -p 6 -x ${hisat2_index} \
           -1 ${trim_out_fq1} \
           -2 ${trim_out_fq2} \
           -S ${align_dir}/${sample}.sam; } 2>> ${RESULTS}/optim2_hisat2_align_run.txt

    # RUN featureCounts QUANT     
    source /app1/ebenv
    module unload HISAT2/2.2.1-gompi-2023a
    module load Subread/2.0.4-GCC-11.3.0
    echo "Starting featurecounts quant for $sample_id"
    { time featureCounts -p --countReadPairs \
                  -a ${gtf} \
                  -T 6 \
                  -t exon \
                  -g gene_id \
                  -M \
                  -o ${quant_dir}/${sample}_counts.txt \
                  ${sam}; } 2>> ${RESULTS}/optim2_subread_quant_run.txt   

    wait
) 2>> ${RESULTS}/${sample_id}_optim2_run.txt
EOF

done


# 5. Nextflow

In [ ]:
# main pbs script for qsub
#!/bin/bash
#PBS -N nextflow_pipeline
#PBS -o nextflow_output.log
#PBS -e nextflow_error.log
#PBS -l walltime=24:00:00
#PBS -l select=1:ncpus=12:mem=24gb

# Load required modules
source /app1/ebenv
module load Nextflow

# Change to the directory where the job was submitted
cd $PBS_O_WORKDIR

mkdir -p /hpctmp/e1324266/5004/nf/results

# Run Nextflow
{ time nextflow run main.nf -c nextflow.config -profile module; } 2> /hpctmp/e1324266/5004/nf/results/nf_total_time.txt


In [ ]:
# Nextflow.config file
// CLI
dag.enabled = true
report.enabled = true
timeline.enabled = true

// Set up scheduler
process {
    process {
    executor = 'pbspro'
    clusterOptions = '-V'
    }
}

// Set up resources
process {

    withLabel: low {
                cpus = 4
                memory = 8.GB
    }

    withLabel: high {
                cpus = 6
                memory = 12.GB
    }
}

//configuration_profiles

profiles {

    module {
                process {

                        withName: FASTQC {
                                beforeScript = '''
                                source /app1/ebenv
                                module load FastQC/0.12.1-Java-11
                        '''
                        }

                        withName: TRIM {
                                beforeScript = '''
                                source /app1/ebenv
                                module load fastp/0.23.2-GCC-11.3.0 
                        '''
                        }

                        withName: HISAT2 {
                                beforeScript = '''
                                source /app1/ebenv
                                module load HISAT2/2.2.1-gompi-2023a
                        '''
                        }

                        withName: FEATURECOUNTS {
                                beforeScript = '''
                                source /app1/ebenv
                                module load Subread/2.0.4-GCC-11.3.0
                        '''
                        }
                }

    }
}

In [ ]:
nextflow.enable.dsl=2

params.input_dir = "/hpctmp/e1324266/5004/raw/download/fastq_files"
params.output_dir = "/hpctmp/e1324266/5004/nf/results"
params.fastqc_dir = "${params.output_dir}/00-fastqc"
params.trim_dir = "${params.output_dir}/01-trim"
params.align_dir = "${params.output_dir}/02-align"
params.quant_dir = "${params.output_dir}/03-quant"

params.ref_dir = "/hpctmp/e1324266/5004/ref/paper"
params.annotation = "${params.ref_dir}/gencode.v18.annotation.gtf"
params.genome = "${params.ref_dir}/GRCh37.p12.genome.fa"
params.transcript = "${params.ref_dir}/gencode.v18.pc_transcripts.fa"
params.index = "/hpctmp/e1324266/5004/ref/hisat2_index/genome"


// Define processes
process FASTQC {
    publishDir params.fastqc_dir, mode:'copy'

    label 'low'
    
    input:
    tuple val(sample_id), path(read1), path(read2)

    output:
    path("*")

    script:
    """
    fastqc --threads ${task.cpus} $read1 $read2
    """
}

process TRIM {
    // Output to params.trim_dir directory
    publishDir params.trim_dir, mode: 'copy'
    
    label 'high'
    
    input:
    tuple val(sample), path(read1), path(read2)

    output:
    tuple val(sample), path("${sample}_trimmed_1.fq.gz"), path("${sample}_trimmed_2.fq.gz")

    script:
    """
    fastp --thread ${task.cpus} --in1 $read1 --in2 $read2 --out1 ${sample}_trimmed_1.fq.gz --out2 ${sample}_trimmed_2.fq.gz --detect_adapter_for_pe --qualified_quality_phred 20
    """
}

process HISAT2 {
    publishDir params.align_dir, mode: 'copy'
    
    label 'high'
    
    input:
    tuple val(sample), path(read1), path(read2)
    
    output:
    tuple val(sample), path("${sample}.sam")
    
    script:
    """
    hisat2 -p ${task.cpus} -x ${params.index} \
           -1 ${read1} \
           -2 ${read2} \
           -S ${sample}.sam
    """
}

process FEATURECOUNTS {
    // Output to params.quant_dir
    publishDir params.quant_dir, mode: 'copy'
    
    label 'high'
    
    input:
    tuple val(sample), path(file)

    output:
    path "*"

    script:
    """
    featureCounts -p --countReadPairs \
                  -a ${params.annotation} \
                  -T ${task.cpus} \
                  -t exon \
                  -g gene_id \
                  -M \
                  -o ${sample}_counts.txt \
                  ${file}
    """
}

// Define the workflow
workflow {
ch = channel.fromFilePairs("${params.input_dir}/*_{1,2}.fastq.gz",flat: true)
FASTQC(ch)
TRIM(ch)
HISAT2(TRIM.out)
FEATURECOUNTS(HISAT2.out)
}

# 6. References

* https://bmcgenomics.biomedcentral.com/articles/10.1186/s12864-024-10414-y
* https://www.frontiersin.org/journals/plant-science/articles/10.3389/fpls.2021.657240/full
* https://academic.oup.com/nargab/article/6/2/lqae040/7659592